# Transfer Learning from Tennis to Pong

**Neuromatch Reinforcement Learning Project 2023**

__Authors:__ Amirozhan Dehghani, Parmida Pezeshki, Alireza Astane, Sanchit Baweja



# Things to work on

1.   Find the best parameters to train the RL on Pong
2.   Why is the loss so small?
3.   ~~Fix the bug when training on Pong Enviroment~~ (**Resolved**)
4.   Find the best parameters to train the RL on Pong
5.   Clean-up the code for easy training on either enviroments
6.   Write the code to transfer the weights from Pong to Tennis enviroment
7.   Train the model on Pong and transfer it to train on Tennis
8.   Save the results







In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install -y swig build-essential python-dev python3-dev > /dev/null 2>&1

In [ ]:
# @title Install dependencies
!pip install rarfile --quiet
!pip install stable-baselines3[extra] --quiet
!pip install ale-py --quiet
!pip install gym[box2d] --quiet
!pip install pyvirtualdisplay --quiet
!pip install pyglet --quiet
!pip install pygame --quiet
!pip install minigrid --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.4/178.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 9.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 90.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wh

In [ ]:
# Imports
import io
import os
import glob
import pandas as pd
from statistics import mean as MEAN

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import make_grid
from torch.optim.lr_scheduler import StepLR
import math
import cv2
import numpy as np
import random
import base64
import time
from google.colab import drive

import numpy as np
import matplotlib.pyplot as plt

import sys
import gymnasium
sys.modules["gym"] = gymnasium
from itertools import count
from collections import namedtuple, deque

import gymnasium as gym
from gym import spaces
from gym.wrappers.monitoring.video_recorder import VideoRecorder

pygame 2.5.0 (SDL 2.28.0, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Parameter Selection for Model Training


1. Try as many parameter combinations as possible.
  * We might need to grayscale enviroment
  * We might need batch normalization after each Conv layer

3. The parameters, along with the final reward and loss, will be saved in a CSV file.
  * If you want to submit your experiment as an official entry, change the `SAVE` variable to **"Shared"** so we can evaluate the best parameters.


I found a similar code, look at their hyperparameters [GitHub - rl-atari-tennis](https://github.com/leerumor/rl-atari-tennis).



**OLD PARAM**
* ~~Parmida: lr= 0.025 , tau= 0.005, batch-size=512, num_episdodes = 250~~ (not good)  
* ~~Shreya:  lr= 0.025, tau= 0.05, batch-size=512, num_episdodes = 250~~ (not good)
  * Shreya will also run with 3 channel instead of 4 channels  
* ~~Sanchit: lr=0.0025  tau=0.05, batch_size = 512, num_episdodes = 250~~ (not good)
* Alireza: lr=0.0025, tau= 0.005, batch_size = 512, num_episdodes = 250
* Amirozhan: lr= 0.025, tau= 0.5, batch_size = 512, num_episdodes = 250 **(best one so far)**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
# SAVE to choose whether to save the file locally or in a shared folder

SAVE = "Locally" #@param ["Locally","Shared"]
Author = "Amirozhan" #@param ["Amirozhan", "Sanchit", "Shreya", "Parmida", "Alireza", "Noureldin"] {allow-input: true}
num_episodes = 1000 #@param [50,100,500,1000] {type:"raw", allow-input: true}
GAMES = "Pong" #@param ["Tennis","Pong","CarPole-v0"]
BATCH_SIZE = 32 #@param [128,256,512,1024] {type:"raw", allow-input: true}
GAMMA = 0.99 #@param [0.99,1.5,2.0,2.5] {type:"raw", allow-input: true}
DECAY_TYPE = "Exponential" #@param ["Exponential","Linear","Memorical"]
EPS_START = 1 #@param [0.5,0.9,2,2.5,3] {type:"raw", allow-input: true}
EPS_END = 0.01 #@param [0.01,0.04,0.05,0.1] {type:"raw", allow-input: true}
EPS_DECAY = 700 #@param [100,500,1000,1500] {type:"raw", allow-input: true}
TAU = 1e-3 #@param [0.00005,0.0005,0.005,0.05] {type:"raw", allow-input:
LR = 1e-4 #@param [1e-6,1e-5,1e-4,1e-3] {type:"raw", allow-input: true}
MIN_MEMORY_LEN = 10000 #@param
MEMORY_SIZE = 100000 #@param
STEP_SIZE = 300 #@param {type:"raw", allow-input: true}
plot_every = 20 #@param [5,20,50,100] {type:"raw", allow-input: true}
numOfLastActions = 5 #@param [5,10,15,20] {type:"raw", allow-input: true}
UPDATE_FREQ = 1000#@param [5,10,15,20] {type:"raw", allow-input: true}
seed = 42 #@param {type:"integer"}


my_param={
    "Empty" : "",
    "time": None,
    "EPOCHS": [num_episodes],
    "MEMORY_SIZE": [MEMORY_SIZE],
    "MIN_MEMORY_LEN":[MIN_MEMORY_LEN],
    "Final_Loss" : None,
    "Final_Reward" : None,
    "UPDATE_FREQ" : [UPDATE_FREQ],
    "STEP_SIZE": [STEP_SIZE],
    "Author" : [Author],
    "Device" : [device],
    "BATCH_SIZE" : [BATCH_SIZE],
    "GAMES" : [GAMES],
    "GAMMA" : [GAMMA],
    "DECAY_TYPE" : [DECAY_TYPE],
    "EPS_START": [EPS_START],
    "EPS_END" : [EPS_END],
    "EPS_DECAY" : [EPS_DECAY],
    "TAU":[TAU],
    "LR":[LR],
          }

In [ ]:
def _to_save (SAVE):
  filename = "summary.csv"
  if SAVE == "Locally":
      save_directory = "./Saved_Params"
      os.makedirs(save_directory, exist_ok=True)
      filepath = os.path.join(save_directory, filename)
  elif SAVE == "Shared":
      drive.mount('/content/drive')
      folder_path = '/content/drive/MyDrive/NeuroMartch_Academy '
      if not os.path.exists(folder_path):
        raise ValueError("Please move the NeuroMartch_Academy to the MyDrive")
      filepath = os.path.join(folder_path, filename)
  return filepath

In [ ]:
def Save_as_csv(SAVE,summary_data):
    filepath = _to_save(SAVE)
    summary_df = pd.DataFrame.from_dict(summary_data)

    if os.path.exists(filepath):
        # If the file exists, load the existing DataFrame and append the new row
        existing_df = pd.read_csv(filepath, index_col=0)
        updated_df = existing_df.append(summary_df, ignore_index=True)

    else:

        updated_df = summary_df

    updated_df.to_csv(filepath, index=False)

**Memory Replay We might use it?**

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info
    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs
class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,) + env.observation_space.shape, dtype=np.uint8)
        self._skip = skip

    def reset(self):
        return self.env.reset()

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info, _ = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        Expects inputs to be of shape num_channels x height x width.
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0] * k, shp[1], shp[2]), dtype=np.uint8)

    def reset(self,**kwargs):
        ob = self.env.reset(**kwargs)
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info,_ = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0


class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers."""
        self._frames = frames

    def __array__(self, dtype=None):
        out = np.concatenate(self._frames, axis=0)
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._frames)

    def __getitem__(self, i):
        return self._frames[i]


class PyTorchFrame(gym.ObservationWrapper):
    """Image shape to num_channels x height x width"""

    def __init__(self, env):
        super(PyTorchFrame, self).__init__(env)
        shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(shape[-1], shape[0], shape[1]), dtype=np.uint8)

    def observation(self, observation):
        return np.rollaxis(observation, 2)
class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work.
        Expects inputs to be of shape height x width x num_channels
        """
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255,
                                            shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]
class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)
class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.integers(1, self.noop_max + 1)  # pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ , _= self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

In [ ]:
def resize_frame(state):
    state = state[30:-12,5:-4]
    state = np.average(state,axis = 2) #Grayscaling the depth channel
    state = cv2.resize(state,(84,84),interpolation = cv2.INTER_NEAREST)
    state = np.ascontiguousarray(state, dtype=np.float32) / 255
    return state

In [ ]:
def get_epsilon(DECAY_TYPE,i_episode):
  if DECAY_TYPE == "Exponential":
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-2. * i_episode / EPS_DECAY)
  elif DECAY_TYPE == "Linear":
     eps_threshold = max(EPS_START - (EPS_START - EPS_END) * (i_episode / EPS_DECAY),EPS_END)
  #new
  elif DECAY_TYPE == "Memorical":
     eps_threshold = max(1/(getVarOfLastActions() + 1/EPS_START),EPS_END)  # 5 can be changed
  return eps_threshold


In [ ]:

#new  #its dirty but im realy not familiar with deques

def getVarOfLastActions():
  lastActions = np.zeros(numOfLastActions)
  try:
    for i in range(numOfLastActions):
      lastActions[i] = int(memory.memory[-i][1][0][0])
  finally:
    return np.var(lastActions)
"""

def getVarOfLastActions():
  start = time.time()
  last_actions = torch.Tensor([transition.action[0] for transition in memory.memory][-numOfLastActions:])
  if len(last_actions) == numOfLastActions:
    end = time.time()
    print(end-start)
    return torch.var(last_actions,dim=0,keepdim=True)
  else:
    return 1
  """


'\n\ndef getVarOfLastActions():\n  start = time.time()\n  last_actions = torch.Tensor([transition.action[0] for transition in memory.memory][-numOfLastActions:])\n  if len(last_actions) == numOfLastActions:\n    end = time.time()\n    print(end-start)\n    return torch.var(last_actions,dim=0,keepdim=True)\n  else:\n    return 1\n  '

In [ ]:
env_name = f"PongNoFrameskip-v4"
env = gym.make(env_name,render_mode="rgb_array")
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)
np.random.seed(seed)
random.seed(seed)
env.seed(seed)
env = NoopResetEnv(env, noop_max=30)
env = MaxAndSkipEnv(env, skip=4)
env = EpisodicLifeEnv(env)
env = FireResetEnv(env)
env = WarpFrame(env)
env = PyTorchFrame(env)
env = ClipRewardEnv(env)
env = FrameStack(env, 4)


State shape:  (210, 160, 3)
Number of actions:  6


In [ ]:
class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()

        self.conv1 = nn.Conv2d(n_observations, 16, kernel_size=(8,8),stride=4)
        #self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32,64,kernel_size=(4,4),stride = 2)
        #self.bn2 = nn.BatchNorm2d(64)
        #self.conv3 = nn.Conv2d(64,64,kernel_size=(3,3),stride =1)
        #self.bn3 = nn.BatchNorm2d(64)
        self.layer1 = nn.Linear(3136, 256)
        self.layer2 = nn.Linear(256, n_actions)
    def forward(self, x):

        #x = x.unsqueeze(1) #Used for GrayScale
        #x = x.permute(0,3,1,2)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        #x = F.relu(self.conv3(x))
        x = x.reshape(x.size(0), -1)
        x = F.relu(self.layer1(x))
        out = self.layer2(x)
        return out


In [ ]:
n_actions = env.action_space.n


#n_observations = 3 #change when running RGB
n_observations = 4


policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

#optimizer = optim.Adam(policy_net.parameters(), lr=LR)
# Add AdamW: Small error in the implementation of the Adam.
#optimizer = optim.SGD(policy_net.parameters(), lr=LR)
optimizer = optim.RMSprop(policy_net.parameters(), lr=LR, alpha=0.9, eps=1e-02, momentum=0.0)
# make Gamma larger
#Should work with no scheduler
scheduler = StepLR(optimizer, step_size=STEP_SIZE, gamma=0.1)

memory = ReplayMemory(MEMORY_SIZE)

# Take output of the policy net state, devide it by TAU, apply softmax. and then decrease the TAU
def select_action(state):
    sample = random.random()

    eps_threshold= get_epsilon(DECAY_TYPE,i_episode)

    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state).argmax().view(1, 1)
    else:
        return torch.tensor([[env.action_space.sample()]], device=device, dtype=torch.long)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:

class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()

        self.conv1 = nn.Conv2d(n_observations, 32, kernel_size=(8,8),stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.dropout = nn.Dropout(p=0.3)
        self.conv2 = nn.Conv2d(32,64,kernel_size=(4,4),stride = 2)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(p=0.3)
        self.conv3 = nn.Conv2d(64,64,kernel_size=(3,3),stride =1)
        self.bn3 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(p=0.3)
        self.layer1 = nn.Linear(3136, 512)
        self.layer2 = nn.Linear(512, n_actions)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout(x)
        x = x.reshape(x.size(0), -1)
        x = F.relu(self.layer1(x))
        out = self.layer2(x)
        return out


In [ ]:
model = DQN(n_observations, n_actions)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Print the model summary
summary(model, input_size=(n_observations, 84, 84))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 20, 20]           8,224
       BatchNorm2d-2           [-1, 32, 20, 20]              64
           Dropout-3           [-1, 32, 20, 20]               0
            Conv2d-4             [-1, 64, 9, 9]          32,832
       BatchNorm2d-5             [-1, 64, 9, 9]             128
           Dropout-6             [-1, 64, 9, 9]               0
            Conv2d-7             [-1, 64, 7, 7]          36,928
       BatchNorm2d-8             [-1, 64, 7, 7]             128
           Dropout-9             [-1, 64, 7, 7]               0
           Linear-10                  [-1, 512]       1,606,144
           Linear-11                    [-1, 6]           3,078
Total params: 1,687,526
Trainable params: 1,687,526
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.11
Forw

In [ ]:
%%bash
pip install torchsummary
pip install torchviz graphviz


  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=98ddf64b859a259f2e53bfbbc989806388697030d18509b5d65207457e025ab3
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchviz import make_dot
from graphviz import Digraph
from torchsummary import summary

In [ ]:
class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()

        self.conv1 = nn.Conv2d(n_observations, 32, kernel_size=(8, 8), stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout(p=0.3)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=(4, 4), stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout(p=0.3)

        self.conv3 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.dropout3 = nn.Dropout(p=0.3)

        self.layer1 = nn.Linear(3136, 512)
        self.layer2 = nn.Linear(512, n_actions)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = x.reshape(x.size(0), -1)
        x = F.relu(self.layer1(x))
        out = self.layer2(x)
        return out


In [ ]:
# Assuming n_observations and n_actions are already defined
model = DQN(n_observations, n_actions)
example_input = torch.randn(1, n_observations, 84, 84)


In [ ]:
# Pass the example input through the model
output = model(example_input)

# Create a computation graph visualization
dot = make_dot(output, params=dict(model.named_parameters()))

# Save the graph to a file (optional)
dot.format = 'png'
dot.render('dqn_model_graph', view=False)


/usr/local/lib/python3.10/dist-packages/torchviz/dot.py:65: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(torch.__version__) < LooseVersion("1.9") and \


'dqn_model_graph.png'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchviz import make_dot

class DQN(nn.Module):
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()

        self.conv1 = nn.Conv2d(n_observations, 32, kernel_size=(8, 8), stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout(p=0.3)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=(4, 4), stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout2 = nn.Dropout(p=0.3)

        self.conv3 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.dropout3 = nn.Dropout(p=0.3)

        self.layer1 = nn.Linear(3136, 512)
        self.layer2 = nn.Linear(512, n_actions)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = x.reshape(x.size(0), -1)
        x = F.relu(self.layer1(x))
        out = self.layer2(x)
        return out

# Assuming n_observations and n_actions are already defined
model = DQN(n_observations, n_actions)
example_input = torch.randn(1, n_observations, 84, 84)

# Create a computation graph visualization without extending the convolutional layers
dot = make_dot(model(example_input), params=dict(model.named_parameters()), show_attrs=True, show_saved=True)

# Set the background color to black and render the graph
dot.format = 'png'
dot.attr(bgcolor='grey')
dot.render('dqn_model_graph_dark', view=True)


/usr/local/lib/python3.10/dist-packages/torchviz/dot.py:65: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(torch.__version__) < LooseVersion("1.9") and \


'dqn_model_graph_dark.png'

In [ ]:
def optimize_model():
    if len(memory) < MIN_MEMORY_LEN:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)


    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])


    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)


    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():

        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    #criterion = nn.CrossEntropyLoss()
    #criterion = nn.MSELoss()

    #loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    #torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    #torch.nn.utils.clip_grad_value_(policy_net.parameters(), 1)

    optimizer.step()
    #scheduler.step()
    return loss

In [ ]:
#new
action_labels = {
    0: 'NOOP',
    1: 'FIRE',
    2: 'RIGHT',
    3: 'LEFT',
    4: 'RIGHTFIRE',
    5: 'LEFTFIRE'
}
def doplot(num):
    #w=target_net.conv1.weight.detach().cpu()
    #w=torch.nn.functional.normalize(w, dim=(1))

    #img=make_grid(w,nrow=8,padding=2,normalize=True)
    #img_numpy=img.permute(1,2,0).numpy()

    last_actions = [int(transition.action[0]) for transition in memory.memory][-t:]
    action_counts = [last_actions.count(action) for action in action_labels.keys()]



    fig.suptitle(f'DQN Model on {GAMES}')
    ax3.cla()
    ax1.set_yscale('log')
    ax1.plot(np.arange(len(average_loss)),average_loss)
    ax1.set_ylabel("Average_Loss")
    ax2.plot(np.arange(len(main_reward)),main_reward,color='red')
    ax2.set_ylabel("Reward")
    ax2.set_ylim(-22, 22)
    ax3.bar(list(action_labels.values()), action_counts, width=0.6, color='purple')
    ax3.tick_params(axis='x', labelsize=8, rotation=45)
    ax3.set_ylabel('Counts')
    ax4.plot([get_epsilon(DECAY_TYPE,i) for i in range(i_episode)])
    ax4.set_ylabel(f'{DECAY_TYPE}_Decay_Epsilon')
    plt.tight_layout(rect=[0, 0.1, 1, 1])
    fig.savefig(f"{num}.png")


In [ ]:
#since training models takes so much time. We can save trained model using below functions
#Sample Usage
#modelfile=save_model(target_net,my_param)
#load_model(model,modelfile)
def save_checkpoint(state, modelname):
    torch.save(state, modelname)

def save_model(model,my_param):
  save_checkpoint({
  'epoch': i_episode,
  'state_dict': model.state_dict(),
  'optimizer' : optimizer.state_dict(),},
  modelname="model_{}_ep{}_BS{}_LR{}_tau{}.pth".format(my_param["Author"],my_param["EPOCHS"],my_param["BATCH_SIZE"],my_param["LR"],my_param["TAU"]))

def load_model(model,modelfile):
  checkpoint=torch.load(modelfile)
  model.load_state_dict(checkpoint['state_dict'])
  optimizer =  modelfile[['optimizer']]
  i_episode = modelfile[['epoch']]
  LR = optimizer.state_dict()["param_groups"][0]["lr"]
  return model ,optimizer, i_episode, LR

In [ ]:
timeframe = 0
running_loss = []
running_reward = []
average_reward = []
average_loss = []
main_reward = []
x_ax = np.arange(num_episodes)

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(12, 6))

for i_episode in range(num_episodes):
    total_reward = 0
    print(f"episode: {i_episode}")
    start_time = time.time()
    scheduler.step()
    # Initialize the environment and get it's state

    state = env.reset()

    #state = resize_frame(old_state)
    #state = np.stack((state, state, state, state))
    #state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    for t in count():
        action = select_action(state)
        old_observation, reward, terminated, truncated, _ = env.step(action.item())
        observation = resize_frame(old_observation)
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated
        state = state.squeeze(0)

        if terminated:

            next_state = torch.stack((state[0], state[1], state[2],state[3])).unsqueeze(0)

        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device)
            next_state = torch.stack((next_state, state[0], state[1], state[2])).unsqueeze(0)

        # Move to the next state


        state = next_state

        memory.push(state, action, next_state, reward)

        # Perform one step of the optimization (on the policy network)
        loss = optimize_model()
        if loss == None:
          continue
        else:

          running_loss.append(float(loss.detach().cpu().numpy()))

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        if timeframe % UPDATE_FREQ == 0:
          print("Updating the Target Network")
          target_net_state_dict = target_net.state_dict()
          policy_net_state_dict = policy_net.state_dict()
          for key in policy_net_state_dict:
              target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
          target_net.load_state_dict(target_net_state_dict)
        total_reward += float(reward.detach().cpu().numpy())

        running_reward.append(total_reward)
        timeframe = timeframe + 1

        if done:

          main_reward.append(np.mean(running_reward))
          running_reward.clear()
          break

    avg_loss = np.mean(running_loss)
    average_loss.append(avg_loss)
    #new
    if i_episode % plot_every ==0 :
      doplot(i_episode)
    elif i_episode % 530 == 0: #change 530 to any episodes you want
      save_model(target_net,my_param)



#new
doplot(num_episodes)
end_time = time.time()
duration = end_time - start_time
print('Complete')
print(f"It takes {duration:.2f} for {num_episodes}")

my_param['Final_Loss'] = f"{average_loss[-1]:.3g}"
my_param["Final_Reward"] = main_reward[-1]
my_param['time'] =f"{duration:.2f}"
Save_as_csv(SAVE,my_param)



Linear Decay See how the Epsilon changes as a function of Episodes

In [ ]:
target_net.conv1.weight[:,1:3].shape

In [ ]:
epsilon_by_epsiode = lambda steps_done: max(EPS_START - (EPS_START - EPS_END) * (steps_done / EPS_DECAY),EPS_END)


plt.plot([epsilon_by_epsiode(i) for i in range(num_episodes)])

Exponential Decay
See how the Epsilon changes as a function of Episodes

In [ ]:
epsilon_by_epsiode = lambda steps_done: EPS_END + (EPS_START - EPS_END) * math.exp(-2 * steps_done / EPS_DECAY)
plt.plot([epsilon_by_epsiode(i) for i in range(500)])


In [ ]:
# @title Play Video function
from IPython.display import HTML
from base64 import b64encode
from pyvirtualdisplay import Display

# create the directo  ry to store the video(s)
os.makedirs("./video/ALE/", exist_ok=True)
#display = Display(visible=False, size=(1400, 900))
#_ = display.start()

"""
Utility functions to enable video recording of gym environment
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f''

In [ ]:
env = gym.make(env_name, render_mode="rgb_array")
vid = VideoRecorder(env, path=f"video/{env_name}_learned.mp4")
env = FireResetEnv(env)
#env = MaxAndSkipEnv(env)
old_state = env.reset()
state = resize_frame(old_state)

state = np.stack((state, state, state, state))

state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

total_reward = 0
done = False
while not done:
  frame = env.render()
  vid.capture_frame()

  action = select_action(state)

  old_observation, reward, done, truncated, _ = env.step(action.item())
  observation = resize_frame(old_observation)

  total_reward += reward
vid.close()
env.close()
print(f"\nTotal reward: {total_reward}")

# show video
html = render_mp4(f"video/{env_name}_learned.mp4")
HTML(html)

Exponential decay


### saving and loading model


In [ ]:
torch.save(model, pathfile)